In [26]:
from torch.utils import data
from torch.nn.utils import rnn

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np

In [2]:
import networkx as nx

In [3]:
with open('../Dataset/Cora/cora.cites') as fs:
    for line in fs:
        temp = list(map(int,line.strip().split()))
        print(temp[0],temp[1])
        break

35 1033


In [4]:
# construct network
G = nx.Graph()
count = 0
node_list = {} # assign a id of 0 - n (n nodes...)
with open('../Dataset/Cora/cora.cites') as fs:
    for line in fs:
        u,v = map(int,line.strip().split())
        if u not in node_list:
            node_list[u] = count
            count+=1
        if v not in node_list:
            node_list[v] = count
            count+=1
        G.add_edge(node_list[u],node_list[v])    


In [5]:
# obtaining node features...
feature = [[] for i in range(len(node_list))]
label = {}
with open('../Dataset/Cora/cora.content') as fs:
    for line in fs:
        temp = line.strip().split()
        node = node_list[int(temp[0])]
        label = temp[-1]
        feature[node] = list(map(int,temp[1:-1]))

In [6]:
feature = torch.FloatTensor(feature)

In [7]:
feature.shape

torch.Size([2708, 1433])

In [8]:
class Encoder(nn.Module):
    def __init__(self,feature,graph,input_size,hidden_size):
        super(Encoder,self).__init__()
        self.feature = feature
        self.graph = graph
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        self.W_1 = nn.Parameter(torch.rand(input_size,hidden_size))
        self.W_2 = nn.Parameter(torch.rand(input_size,hidden_size))
        
        self.U_1 = nn.Parameter(torch.rand(hidden_size))
        self.U_2 = nn.Parameter(torch.rand(hidden_size))
        
        self.softmax = nn.Softmax(dim=0)

        
    def forward(self,node_sample):
        
        C_n_1_u = torch.matmul(feature,self.W_1)
        
        C_n_2_u = torch.matmul(feature,self.W_2)
        
        s_node_repr = torch.Tensor()
        
        for node in node_sample:
            
            nbr_1 = torch.Tensor()
            nbr_2 = torch.Tensor()
            # obtaining the first hop neighbors
            neighbors_1 = list(nx.neighbors(G,node))
            
            # obtaining the second hop neighbors
            neighbors_2 = []
            for n in neighbors_1:
                nbr_1 = torch.cat((nbr_1,C_n_1_u[n].view(1,-1)),dim=0) # getting the vectors of neighbors
                neighbors_2 += list(nx.neighbors(G,n))
            
            neighbors_2 = list(set(neighbors_2))
            
            for n in neighbors_2:
                nbr_2 = torch.cat((nbr_2,C_n_2_u[n].view(1,-1)),dim=0) # getting vectors of two-hop neighbors
            
            # calculate attention weights for 1-hop neighbors
            
            att_wt_1 = self.softmax(torch.sum(nbr_1*self.U_1,dim=1).view(-1,1))
            att_wt_2 = self.softmax(torch.sum(nbr_2*self.U_2,dim=1).view(-1,1))
            
            output = torch.cat((torch.sum(nbr_1*att_wt_1,dim=0),torch.sum(nbr_2*att_wt_2,dim=0)),dim=0).view(1,-1)
            #print(output)
            
            s_node_repr = torch.cat((s_node_repr,output),dim=0)
            
        return s_node_repr    
            

In [9]:
import random
from collections import Counter

In [10]:
def randomWalk(G,node,walk_length = 3,walk_num=10): # finding list of similar nodes
    sim_nodes = Counter()
    for i in range(walk_num):
        l = 0
        c_node = node
        while(l<walk_length):
            n = random.choice(list(nx.neighbors(G,c_node)),1)
            sim_nodes[n]+=1
            c_node = n
            l+=1
    return list(sim_nodes)        

In [11]:
def findSimilarNodes(G):
    similar_nodes = {}
    for node in G.nodes():
        similar_nodes[node] = randomWalk(G,node)
    return similar_nodes 

In [12]:
def getSamples(G,sample_nodes,all_edges): # generates positive and negative samples for a given batch of nodes
    pos_sample_edges = list(filter(lambda x:x[0] in sample_nodes and x[1] in sample_nodes,all_edges))
    neg_sample_size = len(pos_sample_edges)
    neg_sample_edges = []
    i=0
    tries = 2*neg_sample_size # check to keep the number of tries finite
    while i<neg_sample_size:
        u,v = random.sample(sample_nodes,2)
        if (u,v) not in pos_sample_edges and (v,u) not in pos_sample_edges:
            neg_sample_edges.append((u,v))
            i+=1
        tries-=1
        if tries==0:
            break
    return pos_sample_edges,neg_sample_edges        

In [13]:
def createBatches(Graph = G,batch_size = 1,node_list=[],edge_list=[]):
    random.shuffle(node_list)
    num_batches = int(len(node_list)/batch_size)+1
    for i in range(num_batches):
        # get nodes for batches...
        if i<num_batches-1:
            sample_nodes = node_list[i*batch_size:(i+1)*batch_size]
        else:
            sample_nodes = node_list[i*batch_size:]

        # find positive and negative samples for a batch
        pos_sample_edges, neg_sample_edges = getSamples(Graph,sample_nodes,edge_list)
        
        node_dict = {}
        count = 0
        
        for n in sample_nodes:
            node_dict[n] = count
            count+=1
        
        yield pos_sample_edges,neg_sample_edges,sample_nodes,node_dict

In [14]:
def biasedRandomWalk(u,Graph,sample_size): # mix of BFS and DFS
    node_c = 1
    sample = [u]
    curr_node = u
    while node_c<sample_size:
        
        nbrs = list(nx.neighbors(Graph,curr_node))
        
        if random.uniform(0,1)<0.5:
            v = random.choice(nbrs)
            curr_node = v
            sample.append(v)
            node_c+=1
            
        else:
            
            if len(nbrs)< sample_size - node_c:
                sample+= nbrs
                node_c+= len(nbrs)

            else:
                to_add = sample_size - node_c
                sample += nbrs[:to_add]
                node_c += len(nbrs[:to_add])
                
            curr_node = random.choice(nbrs)    
     
    return list(set(sample))

In [15]:
def createBatchesSubgraph(Graph = '',batch_size = '',node_list='',edge_list=''):
    
    num_batches = 2*int(len(node_list)/batch_size)
    
    for _ in range(num_batches):
    
        batch_size_pos = int(0.8*batch_size)
        batch_size_neg = batch_size - batch_size_pos
        u = random.choice(node_list)
        sample_nodes = biasedRandomWalk(u,Graph,batch_size_pos)
        sample_nodes += random.sample(node_list, batch_size-len(sample_nodes))

        sample_nodes = list(set(sample_nodes))
        
        pos_sample_edges, neg_sample_edges = getSamples(Graph,sample_nodes,edge_list)

        node_dict = {}
        count = 0

        for n in sample_nodes:
            node_dict[n] = count
            count+=1

        yield pos_sample_edges,neg_sample_edges,sample_nodes,node_dict

In [16]:
def train(encoder, graph=G, batch_size='', epochs=1, learning_rate=0.001):
    optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()
    node_list = list(G.nodes())
    edge_list = list(G.edges())
    
    for _ in range(epochs):
        
        batch_count = 0
        for pos_sample, neg_sample, sample_nodes,node_dict in \
        createBatchesSubgraph(Graph=G, batch_size=batch_size, node_list=node_list, edge_list=edge_list):
            
    
            batch_count+=1
            node_repr = encoder(sample_nodes)
            
            
            predicted_vector = torch.Tensor()
            true_label = torch.FloatTensor([1 for i in range(len(pos_sample))]+[0 for i in range
                                                                                   (len(neg_sample))]).view(1,-1)
            
            
            i = 0
            for a,b in pos_sample+neg_sample:
                u = node_dict[a]
                v = node_dict[b]
                pred_val = F.logsigmoid(torch.sum(node_repr[u]*node_repr[v])).view(1,1)
                
                predicted_vector = torch.cat((predicted_vector, pred_val),dim=0)
            
            loss = criterion(predicted_vector,true_label)
            
            if batch_count%20==0:
                print('loss: {}, batch no: {}, epoch: {}'.format(loss,batch_count,_))
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

In [17]:
def getRepresentations(encoder,G):
    nodes = list(G.nodes())
    
    node_repr = encoder(nodes)
    
    return node_repr

In [18]:
feature.shape

torch.Size([2708, 1433])

In [19]:
input_size = feature.shape[1]
hidden_size = 100
encoder = Encoder(feature,G,input_size,hidden_size)
batch_size = 50
train(encoder,graph=G,batch_size=batch_size)

loss: 0.5, batch no: 20, epoch: 0
loss: 0.5, batch no: 40, epoch: 0
loss: 0.5, batch no: 60, epoch: 0
loss: 0.5, batch no: 80, epoch: 0
loss: 0.5, batch no: 100, epoch: 0


In [20]:
from torchviz import make_dot

In [32]:
sample_nodes = random.sample(list(G.nodes()),1)

In [ ]:
make_dot(encoder(sample_nodes))

In [34]:
sample_nodes

[351]

In [35]:
len(list(nx.neighbors(G,351)))

2

In [38]:
list(nx.neighbors(G,351))

[346, 1194]

In [40]:
nbr2 = list(nx.neighbors(G,346)) + list(nx.neighbors(G,1194))

In [41]:
len(nbr2)

55

In [42]:
len(set(nbr2))

51